In [ ]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [ ]:
#importo csv
df_reviews = pd.read_csv('data/user_reviews_clean.csv', encoding='utf-8')
df_games = pd.read_csv('data/steam_games_clean.csv', encoding='utf-8')
df_items = pd.read_csv('data/user_items_clean.csv', encoding='utf-8')

In [ ]:
# Se hace un subconjunto con los que serán necesarios para la consulta
contenido_free_items = df_games[['price', 'release_year', 'developer', 'id']]
# Se renombra la columna para luego poder unirla con items
contenido_free_items = contenido_free_items.rename(columns={'id':'item_id'})

In [ ]:
contenido_free_items.head(2)

In [ ]:
# Se toman las columnas que serán necesarias
df_price = df_games[['price', 'id']]
# Se renombra la columna para luego poder unirla con items
df_price = df_price.rename(columns={'id':'item_id'})

In [ ]:
# Se toman las columnas que serán necesarias
df_price_items = df_items[['items_count', 'user_id', 'item_id']]
df_price_items.head(2)

In [ ]:
# Se hace un merge entre las dos tablas anteriores
df_price_items = df_price_items.merge(df_price, on='item_id',how='left')
df_price_items.head()

In [ ]:
# Se hace una revision de nulos para los precios 
df_price_items['price'].isna().sum()

In [ ]:
# Los precios que tienen nulos, como no tienen asignado un precio se imputaran ceros 
df_price_items['price'] = df_price_items['price'].fillna(0)

In [ ]:
df_price_items

In [ ]:
# Ya que se termino con el merge, se puede eliminar la columna ites_id
df_price_items = df_price_items.drop('item_id', axis=1)

In [ ]:
# Ahora sería de utilidad poder agrupar por usuario y saber cuanto es la sumna de price por usuario
df_price_user = df_price_items.groupby('user_id')['price'].sum().reset_index()
df_price_user

In [ ]:
# Se retoman las columnas necesarias
df_items_count = df_price_items[['items_count','user_id']]
# Y se eliminan los duplicados para poder unirlos con el df_price_user
df_items_count = df_items_count.drop_duplicates(subset='user_id', keep='first')

In [ ]:
df_items_count

In [ ]:
# Ya se tiene el mismo numero de filas, se procede a hacer el merge
df_user_data = df_items_count.merge(df_price_user, on='user_id', how='right')
df_user_data

In [ ]:
# Se seleccionan las que serán utiles para Horas por usuario
df_user_playtime = df_items[['playtime_forever', 'user_id', 'item_id']]
df_user_playtime.head()

In [ ]:
# Se va a necesitar tambien items y genero
df_items_genre = df_games[['genres','id','release_year']]
df_items_genre = df_items_genre.rename(columns={'id':'item_id'})
df_items_genre

In [ ]:
# Se hace un merge de los data frames anteriores
df_user_time_genre = df_user_playtime.merge(df_items_genre, on='item_id')
df_user_time_genre

In [ ]:
# Se hace la agrupacion 
df_user_time_genre = df_user_time_genre.groupby(['genres', 'user_id', 'release_year'])['playtime_forever'].sum().reset_index()
# Se convierte a horas el tiempo jugado
df_user_time_genre['playtime_hrs'] = df_user_time_genre['playtime_forever']/60
# Se elimina la columna anterior de tiempo jugado
df_user_time_genre = df_user_time_genre.drop('playtime_forever', axis=1)
df_user_time_genre

In [ ]:
# Se cargan los datasets con las columnas que serán necesarias
df_dev = df_games[['developer','id','release_year']]
df_items_conect = df_items[['item_id','user_id']]
df_reviews_sentiment = df_reviews[['sentiment_analysis','reviews_recommend','user_id']]

In [ ]:
# Se prepara para hacer el merge
df_dev = df_dev.rename(columns={'id':'item_id'})

In [ ]:
# Se aplica el merge sobre items
df_dev_items = df_dev.merge(df_items_conect,on='item_id')

In [ ]:
df_dev_items

In [ ]:
# se procede a unir con el ultimo dataframe
df_reviews_dev = df_reviews_sentiment.merge(df_dev_items,on='user_id')

In [ ]:
#reviso duplicados
df_reviews_dev.duplicated().sum()

In [ ]:
#elimino duplicados
df_reviews_dev.drop_duplicates(inplace=True)

In [ ]:
#vulvo a contarduplicados
df_reviews_dev.duplicated().sum()

In [ ]:
df_reviews_dev

In [ ]:
# Se elimina la columna que sirvio como union
df_reviews_dev = df_reviews_dev.drop(columns='item_id', axis=1)

In [ ]:
df_reviews_dev.columns

In [ ]:
# Ahora solo dejamos los registros con review igual a true y con analisis de sentimiento = 2
df_best_developer_year = df_reviews_dev[(df_reviews_dev['reviews_recommend'] == True) & (df_reviews_dev['sentiment_analysis'] == 2)]

In [ ]:
df_best_developer_year.head()

In [ ]:
#cambio a tipo int release_year
df_best_developer_year = df_best_developer_year.copy()
df_best_developer_year['release_year'] = df_best_developer_year['release_year'].astype(int)

In [ ]:
df_best_developer_year.head()

In [ ]:
# Se crea el data frame para developer_review_analysis llamado: df_dev_rev_analysis
df_dev_rev_analysis = df_reviews_dev[['developer','sentiment_analysis']]
df_dev_rev_analysis

In [ ]:
df_dev_rev_analysis['sentiment_analysis'].nunique()

In [ ]:
# convierto a parquet contenido_free_items
contenido_free_ruta = 'datos/contenido_free_items.parquet'
contenido_free_items.to_parquet(contenido_free_ruta, index=False)

In [ ]:
# convierto a parquet df_user_data
user_data_ruta = 'datos/user_data_price.parquet'
df_user_data.to_parquet(user_data_ruta, index=False)

In [ ]:
#convierto a parquet user_for_genre
user_time_genre_ruta = 'datos/user_for_genre.parquet'
df_user_time_genre.to_parquet(user_time_genre_ruta, index=False)

In [ ]:
# convierto a parquet best_developer_year
developer_year_ruta = 'datos/best_developer_year.parquet'
df_best_developer_year.to_parquet(developer_year_ruta, index=False)

In [ ]:
# convierto a parquet developer_review_analysis
dev_rev_analysis_ruta = 'datos/dev_rev_analysis.parquet'
df_dev_rev_analysis.to_parquet(dev_rev_analysis_ruta, index=False)

In [ ]:
#convierto a parquet user_reviews_clean
user_reviews_ruta = 'datos/user_reviews_clean.parquet'
df_reviews.to_parquet(user_reviews_ruta, index=False)